# Who's the Better Carry: ADC or Midlaner?

**Name(s)**: Katelyn Villamin and Nancy Shen

**Website Link**: (your website link)

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

import plotly.express as px
pd.options.plotting.backend = 'plotly'

from dsc80_utils import *

## Step 1: Introduction

In [2]:
# TODO

Which role carries the most in winning teams: ADCs or Midlanders

Things to consider: 
- deaths per minute
- kills per minute
- KDA
- cs per minute (gold)
- Winning teams only 
- double/triple/quadra/penta kills 

## Step 2: Data Cleaning and Exploratory Data Analysis

In [3]:
# TODO
league_fp = Path('data') / '2022_LoL_esports_match_data_from_OraclesElixir.csv'
league_uncleaned = pd.read_csv(league_fp, low_memory=False)
#isolate only the winning team stats
league = league_uncleaned[league_uncleaned['result'] == 1]

#get rid of unneccessary columns
columns_needed = ['position', 'kills', 'deaths', 'assists', 'doublekills', 'triplekills', 'quadrakills', 'pentakills', 'damagetochampions', 'dpm', 'earnedgold', 'earned gpm']
league = league[columns_needed]

#change all the position column values that are 'bot' to 'adc' for clarity

league['position'] = league['position'].str.replace('bot', 'adc')

#make all position column entries lower case and getting only the 'bot' and 'mid' positions
league = league[(league['position'].str.lower() == 'adc') | (league['position'].str.lower() == 'mid')]

#change column names to more appropriate ones
league.rename(columns={"damagetochampions": "total damage dealt", "earnedgold": "total gold earned", "earned gpm": "gpm"}, inplace=True)

| Column | Description|
| --- | --- |
| `'position'` | The role of the player in the game |
| `'kills'` | Total number of kills a player has in a game |
| `'deaths'` | Total number of times a player dies in a game |
| `'assists'` | Total number of assits a player has in a game |
| `'doublekills'` | Total number of double kills (killing two opponent players consequtivley) per game |
| `'triplekills'` | Total number of triple kills (killing three opponent players consequtivley) per game |
| `'quadrakills'` | Total number of quadra kills (killing four opponent players consequtivley) per game |
| `'pentakills'` | Total number of penta kills (killing all opponent players consequtivley) per game |
| `'total damage dealt'` | Total damage dealt to enemy champions by player|
| `'dpm'` | Average damage dealt to enemy champions per minute by player|
| `'total gold earned'` | Total gold earned by player in a game (excludes starting gold and inherent gold generation) |
| `'gpm'` | Average gold earned per minute by player (excludes starting gold and inherent gold generation) |

**Univariate Analysis**

distribution of kills 

distribution of total damage dealt 

each graph will have two bell curves one for mid one for bot

In [19]:

mid = league[league['position'] == 'mid']['kills'].value_counts()
adc = league[league['position'] == 'adc']['kills'].value_counts()
labels = ['mid', 'adc']
data = [mid, adc]

df = pd.DataFrame()
df['adc'] = adc
df['mid'] = mid
df['mid'] = df['mid'].fillna(0)
df = df.reset_index()
df.rename(columns={"index":"num kills"}, inplace=True)
df
plt.hist(x =df['num kills'], y= df['adc'], label = 'adc')
# plt.hist(x =df['num kills'], y =df['mid'], label = 'mid')
plt.show()


c:\Users\nancy\miniforge3\envs\dsc80\lib\site-packages\numpy\core\fromnumeric.py:43: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.



ValueError: setting an array element with a sequence.

<Figure size 1000x500 with 1 Axes>

In [5]:
# # df = pd.DataFrame()
# # df['num kills'] = league.groupby(['kills', 'position'])['deaths'].count()
# # df = df.reset_index()
# univariate = px.histogram(df, x = 'kills', y = 'num kills', color='position', nbins=30)
# univariate.show()

## Step 3: Assessment of Missingness

In [ ]:
# TODO

## Step 4: Hypothesis Testing

In [ ]:
# TODO


## Step 5: Framing a Prediction Problem

In [25]:
# which is a more accurate measure for damage contribution: dpm vs. total damage dealt?

## Step 6: Baseline Model

In [ ]:
# TODO

## Step 7: Final Model

In [ ]:
# TODO

## Step 8: Fairness Analysis

In [ ]:
# TODO